## Error analysis of sequencing data

Here Biopython is imported to parse sequencing data from SAM files that were generated using bwa mem and to use its gc fraction function. \
Numpy is used to help perform calculations on arrays instead of parsing lists. \
Pandas is used to format results into tables and export as csv files.

In [36]:
import Bio
from Bio import SeqIO
from Bio.SeqUtils import gc_fraction
from Bio import Align
import pysam

import numpy as np
import pandas as pd

#### Q-score functions

The function below grabs the index of each hompolymer of at least the minimum size. Then simply gets the quality score of these indexes (homopolymers) to get the average, as well as calculating the overall average q-score. 

In [37]:
def polymer_avg_qscore(recseq, plength = 3, downstream = 1, skip = 2, trim = 0, type = None):
    """
    A function that anaylses the q-score within hompolymers,
    returns the average q-score within homopolymers and the overall average q-score.
    recseq - is a seqrecord object that must contain quality scores
    plength - int, length (equal or grater than) of homopolymers to be detected. Default = 3
    downstram - int, how large a margin in bases to look for. For example, if 0 then LQB must.
                be within the homopolymer, if 1 can be within homopolymer and one bp downstream.
                Default = 1
    skip - int, number of bp to skip at the beginning of the homopolymer when looking for LQB.
           default = 2
    trim - float, percentage (0 to 1) of start and tail of sequence removed prior to counting,
           a value of 0 results in no trimming. Default = 0.1
    type - None, "AT" or "GC", type of homopolymers to look for,
           None searches for all 
    """
    #Trim sequence
    recseq = recseq[int(trim*len(recseq.seq)):int((1-trim)*len(recseq.seq))]
    #Index of beginning of homopolymers
    if type == None:
        idx_list = [idx for idx,i in recseq.seq.search(["A"*plength, "T"*plength, "G"*plength, "C"*plength])]
    elif type == "AT":
        idx_list = [idx for idx,i in recseq.seq.search(["A"*plength, "T"*plength])]
    elif type == "GC":
        idx_list = [idx for idx,i in recseq.seq.search(["G"*plength, "C"*plength])]
    else:
        raise ValueError("type should be None 'AT' or 'GC'")

    if len(idx_list) > 0:
        idx_list.append(idx_list[-1] + 100) #dummy value as the last element in the loop below is ignored.
    new_idx_list = []
    area = []
    cycle = 0 #To distinguish start and end
    # This section here combines hompolymer run indexes
    for i in range(len(idx_list)):
        if i == 0: #skip first index
            continue
            
        if ((idx_list[i-1] + plength) > idx_list[i]):
            if cycle == 0:
                cycle = 1
                area.append(idx_list[i-1])
        else:
            if cycle == 1:
                area.append(idx_list[i-1])
                new = list(range(area[0]+skip,area[1]+plength+downstream))
                new_idx_list.append(new)
                area = []
                cycle = 0
            else:
                new = list(range(idx_list[i-1]+skip, idx_list[i-1]+plength+downstream)) 
                new_idx_list.append(new)

    qscore = 0
    pol_length = 0
    for i in new_idx_list:
        for j in i:
            #catching occurences where a hompolymer occurs at the end of a read
            if j == len(recseq.seq):
                continue
            pol_length += 1
            qscore += recseq.letter_annotations["phred_quality"][j]
    if pol_length == 0:
        return(0, (sum(recseq.letter_annotations["phred_quality"]) / len(recseq.seq)))
    else:
        return ((qscore/pol_length), (sum(recseq.letter_annotations["phred_quality"]) / len(recseq.seq)))

This function analyses the average q-score of a certain windowsize and records the gc content.

In [38]:
def analyse_gc_qscore(recseq, windowsize, step, trim = 0):
    """
    A function that performs a sliding window analysis of GC conent.
    Calculates the GC fraction of each window and its average Q-score
    Returns two lists, the first containing gc fractions and the second
    containing the Q-score.
    recseq - is a seqrecord object that must contain quality scores.
    qscore - int, any q-score equal to or less than this value will be 
             treated as a LQB.
    windowsize - int, the size of the sliding window
    step - int, the number of bases the window slides by through each iteration.
    trim - float, percentage (0 to 1) of start and tail of sequence removed
           prior to counting, a value of 0 results in no trimming. Default = 0.1
    """
    recseq = recseq[int(trim*len(recseq.seq)):int((1-trim)*len(recseq.seq))]
    q_list = recseq.letter_annotations["phred_quality"]

    if windowsize > len(recseq):
        return([(gc_fraction(recseq.seq), (sum(q_list) / len(q_list)))])

    qscores = []
    i = 0
    while (i + windowsize) < len(recseq):

        sequ = recseq[i:i+windowsize]
        gc = gc_fraction(sequ.seq)
        lqb_count = 0
        avgscore = sum(q_list[i:i+windowsize]) / len(sequ)
        qscores.append((gc,avgscore))
        i = i + step

    return(qscores)

In [39]:
def principal_period(s):
    """
    A function that detects if a string in its entirety consists of a repeating pattern.
    Returns a string of the repeating pattern.
    E.g. "ATGATG" returns "ATG", "ATGATGT" returns None
    """
    i = (s+s).find(s, 1, -1)
    return None if i == -1 else s[:i]

def find_repeats_qscore(recseq, repeat_length = 3, ignore = 0, downstream = 1, trim = 0):
    """
    A function that finds the average score of the bases in a repeat and
    a specified number of bases downstream of the repeat.
    Returns the average score across these bases.
    recseq - is a seqrecord object that must contain quality scores.
    repeat_length - the size of the repeating pattern. Default = 3
    ignore - how many bases to ignore at the start of the repeat. Default = 0
             for example, a value of 2, with "ATGATG", only "GATG" is counted.
    downstram - int, how many bases downstream to include in the calculation.
                for example, value of 1 means in "ATGATGC" the C will be taken
                into account while 0 means only the repeat "ATGATG" will be taken
                into account. Default = 1
    trim - float, percentage (0 to 1) of start and tail of sequence removed prior to counting,
           a value of 0 results in no trimming. Default = 0.1
    """

    #Overlapping repeats are treated individually. E.g TATATAATA has TATAT and ATAATA overlapping.
    #Means more emphasis on these regions.
    q_list = recseq.letter_annotations["phred_quality"] 
    sequ = recseq[int(trim*len(recseq.seq)):int((1-trim)*len(recseq.seq))]
    idx_list = []
    previous = ""
    #This section here creates a list of lists of the start and end indexes
    # of each repeat. E.g [[201,206], [353,358]...]
    for i in range(repeat_length): #For each reading frame
        while (i+(repeat_length*2)) < len(sequ):
            x = sequ[i:i+(repeat_length*2)].seq #Sliding window
            pat = principal_period(x)
            if pat is not None:
                if len(pat) == repeat_length: #If pattern present
                    if pat == previous: #If repeat is continuing, edit previous end index.
                        idx_list[-1][1] = idx_list[-1][1] + repeat_length
                    else: #Else add new index range.
                        idx_list.append([i,i+(repeat_length*2)])
                        previous = pat
            
            i += repeat_length

    #NOTE: probably possible to remove the whole sliding window by repeat size and just
    #      have it always increment by 1.
    
    q_scores = []
    for i in idx_list:
        # if statement to avoid out of index range errors
        if (i[1] + downstream) >= len(q_list):
            q_scores = q_scores + q_list[i[0]:i[1]]
        else:
            q_scores = q_scores + q_list[(i[0] + ignore):(i[1] + downstream)]

    if len(q_scores) == 0:
        return 0
    return(sum(q_scores) / len(q_scores))         

-----------

#### Indel/substitution analysis
In contrast to the earlier Q-score analysis, this now relies on SAM files to detect indels, transitions and transversions

This simply checks if a particular string has a polymer (used later). Can also check for only AT or GC polymers

In [40]:
def has_polymer(sequ, length = 3, type = None):
    
    """
    A function that checks if a sequence contains a homopolymer of certain length.
    length - int, minimum length of hompolymer
    type - None, "AT" or "GC", type of homopolymers to look for
           None searches for all 
    """

    if type == None:
        if len(sequ) < length:
            print('Error sequence less than homopolymer length')
            return
        elif sequ.count("A"*length) != 0:
            return True
        elif sequ.count("T"*length) != 0:
            return True
        elif sequ.count("G"*length) != 0:
            return True
        elif sequ.count("C"*length) != 0:
            return True
        else:
            return False
    elif type == "AT":
        if len(sequ) < length:
            print('Error sequence less than homopolymer length')
            return
        elif sequ.count("A"*length) != 0:
            return True
        elif sequ.count("T"*length) != 0:
            return True
        else:
            return False
    elif type == "GC":
        if len(sequ) < length:
            print('Error sequence less than homopolymer length')
            return
        elif sequ.count("G"*length) != 0:
            return True
        elif sequ.count("C"*length) != 0:
            return True
        else:
            return False
    else:
        raise ValueError("Type must be None, 'AT' or 'GC'")

The function below is used because the original biopython parse function doesn't ignore failed alignments and throws an error. \
The failed alignments are removed later.

In [41]:
def parse_iterator(alignments):
    '''
    Some alignments fail to be initiated, which crashes the biopython
    iterator with no recourse. Here we catch these errors and just 
    skip them. Returns a list of all alignments. (Alignment objects)
    alginments - AlignmentIterator Object
    '''
    ret = []
    while True:
        try:
            x = next(alignments)
        except TypeError: #Catch failed initialization
            continue
        except StopIteration: # Raised at end of iterator, stop here.
            break
        else:
            ret.append(x)
    return(ret)

The class below is set up to store results in an easily accessable manner.

In [42]:
class AlignResults:
    """
    A class to store results from alignment analysis.
    """

    def __init__(self, alignment, gaps, ident, mismatches, transitions, transversions, insertions, deletions,
                gap_idx, iden_idx, mis_idx, ts_idx, tv_idx, ins_idx, del_idx):

        self.alignment = alignment
        self.gaps = gaps
        self.identities = ident
        self.mismatches = mismatches
        self.transitions = transitions
        self.transversions = transversions
        self.insertions = insertions
        self.deletions = deletions
        self.index_gaps = gap_idx
        self.index_identities = iden_idx
        self.index_mismatches = mis_idx
        self.index_transitions = ts_idx
        self.index_transversions = tv_idx
        self.index_insertions = ins_idx
        self.index_deletions = del_idx

    def __repr__(self) -> str:
        return(f"AlignResults Class, {self.gaps}, {self.identities}, {self.mismatches}, {self.transitions}, {self.transversions}, {self.insertions}, {self.deletions}")

    def __len__(self):
        return len(self.alignment[0])

The function below is the main function in this analysis. It analyses a sequence, counts indels, transitions and transversions as well as storing the index of their locations in the alignresults class. \
The information stored here is used for further analysis.

In [43]:
def align_analysis(alignment, show_errors = False, cutoff = None):
    """
    A function that performs analyses on an alignment object.
    Creates an AlignResults objects which contain the results.
    Counts insertions, deletions, mismatches (substitutions),
    gaps, indetities, transitions, transversions and their
    indexes.
    """

    #Sometimes an invalid alignment is created, this skips them
    try:
        x = alignment[1]
    except ValueError:
        if show_errors == True:
            print("Invalid alignment object encountered, skipping...")
        return
    ref = alignment[0]
    gen = alignment[1]
    gaps = 0
    identities, mismatches = 0,0
    transitions, transversions = 0,0
    insertions, deletions = 0,0

    gap_idx, iden_idx, mis_idx = [],[],[]
    ts_idx, tv_idx, ins_idx, del_idx = [],[],[],[]

    if len(ref) != len(gen):
        print("Alignments not same length")
        return

    for i in range(len(ref)):
        if ref[i] == "-":
            insertions += 1
            gaps += 1
            gap_idx.append(i)
            ins_idx.append(i)
        elif gen[i] == "-":
            deletions += 1
            gaps += 1
            gap_idx.append(i)
            del_idx.append(i)
        elif ref[i] == gen[i]:
            identities += 1
            iden_idx.append(i)
        elif ref[i] != gen[i]:
            mismatches += 1
            mis_idx.append(i)
            if gen[i] in ["C","T"]:
                if ref[i] in ["C","T"]:
                    transitions += 1
                    ts_idx.append(i)
                else:
                    transversions += 1
                    tv_idx.append(i)
            else:
                if ref[i] in ["A","G"]:
                    transitions += 1
                    ts_idx.append(i)
                else:
                    transversions += 1
                    tv_idx.append(i)

    if cutoff != None:
        if (gaps + mismatches) > (len(ref) * cutoff):
            return
        
    result = AlignResults(alignment, gaps, identities, mismatches, transitions, transversions, insertions, deletions, gap_idx, iden_idx, mis_idx, ts_idx, tv_idx, ins_idx, del_idx)
    return result

This simply gets the flat error rate for each type of error within a sequence.

In [44]:
def flat_error_analysis(alignres, group = True):
    """
    A function that returns the per base transition rate
    transversion rate, insertion rate and deletion rate.
    alignres - An AlignResults object or list containing
               AlignResults objects.
    group - boole, whether or not to group indels together.
            For example "TG---CT" would be 3 gaps if group
            is False, or would be 1 gap if group is True.
    """

    if type(alignres) == AlignResults:
        if group == False:
            size = len(alignres.alignment[0])
            return(alignres.transitions / size, alignres.transversions / size, alignres.insertions / size, alignres.deletions / size)
        else:
            size = len(alignres.alignment[0])
            insert = len(area_creator(alignres.index_insertions))
            delet = len(area_creator(alignres.index_deletions))
            return(alignres.transitions / size, alignres.transversions / size, insert / size, delet / size)
    
    elif type(alignres) == list:
        ts, tv, ins, dels, size = 0,0,0,0,0
        if group == False:
            for i in alignres:
                if i != None:
                    ts += i.transitions
                    tv += i.transversions
                    ins += i.insertions
                    dels += i.deletions
                    size += len(i.alignment[0])
        else:
            for i in alignres:
                if i != None:
                    ts += i.transitions
                    tv += i.transversions
                    ins += len(area_creator(i.index_insertions))
                    dels += len(area_creator(i.index_deletions))
                    size += len(i.alignment[0])
        if size == 0:
            size = 1
        return([ts/size, tv/size, ins/size, dels/size])
        
    else:
        raise TypeError("Must be an AlignResults class object or List of AlignResult class objects")

In [45]:
def indel_size_analysis(alignres, drop_last_insertion = True, drop_limit = 20):
    """
    A function that measures the size of each indel in a sequence.
    Returns two lists, [insertion sizes], [deletion sizes].
    alignres - An AlignResults object or list containing
               AlignResults objects.
    drop_last_insertion - Bool, Ignore the last insertion in a read if its 
                          size is above the drop limit. Default = True
    drop_limit - int, size limit for last insertion
    """

    if type(alignres) == AlignResults:
        ins_sizes, del_sizes = [],[]
        #Here we simply count the sizes of each indel and append it
        #to a list
        for i in area_creator(alignres.index_deletions):
            if len(i) == 1:
                del_sizes.append(1)
            else:
                del_sizes.append((i[1]-i[0])+1)
        for i in area_creator(alignres.index_insertions):
            if len(i) == 1:
                ins_sizes.append(1)
            else:
                ins_sizes.append((i[1]-i[0])+1)
        #If drop is True, we drop the last insertion if it
        #is too large, this is to get rid of large insertions
        #generated by the aligner at the end of reads.
        if drop_last_insertion == True:
            if len(ins_sizes) > 1:
                 if ins_sizes[-1] > drop_limit:
                     del ins_sizes[-1]
        return(ins_sizes, del_sizes)

    elif type(alignres) == list:
        ins_sizes, del_sizes = [],[]
        for align in alignres:
            if align != None:
                for i in area_creator(align.index_deletions):
                    if len(i) == 1:
                        del_sizes.append(1)
                    else:
                        del_sizes.append((i[1]-i[0])+1)
                for i in area_creator(align.index_insertions):
                    if len(i) == 1:
                        ins_sizes.append(1)
                    else:
                        ins_sizes.append((i[1]-i[0])+1)
                if drop_last_insertion == True:
                    if len(ins_sizes) > 0:
                        if ins_sizes[-1] > drop_limit:
                            del ins_sizes[-1]
        return(ins_sizes, del_sizes)

    else:
        raise TypeError("Must be an AlignResults class object or List of AlignResult class objects")

In [46]:
#First find the homopolymer, then error rate within homopolymer and directly post homopolymer.

def homopolymer_finder(alignres, polymer_length = 3, bases = None):
    """
    A function that finds the indexes of all the
    homopolymers in a sequence.
    Returns a list of start and end indexes of each polymer.
    alginres - AlignResults object or a DNA sequence string.
    polymer_length - int, minumum homopolymer length.
    bases - None, "AT" or "GC", type of homopolymers to look for
            None searches for all 
    """

    #Takes in AlignResults object or a pure sequence.
    if type(alignres) == AlignResults:
        seq = alignres.alignment[0]
    else:
        seq = alignres

    polymer_list = []
    prev = False
    memory, i = 0, 0
    while (i+polymer_length-1) < len(seq):
        if has_polymer(seq[i:i+polymer_length], polymer_length, bases) == True:
            if prev == True:
                i+=1
                continue
            else:
                prev = True
                memory = i
        else:
            if prev == True:
                prev = False
                polymer_list.append([memory, i+polymer_length-2])
        i+=1
    #catch any homopolymers that are at the end of the sequence
    if prev == True:
        polymer_list.append([memory, i+polymer_length-2])
    return(polymer_list)

In [47]:
def homopolymer_analysis(alignres, polymer_length = 3, bases = None, specific = False):
    """
    A function that analyses the error rates due to homopolymers.
    It separately analyses error rates within the homopolymer and
    error rate directly one base downstream of the homopolymer.
    Returns two lists, [homopolymer_errors], [post-polymer errors]
    which each contain four variables representing the error rate
    of: transitions, transversions, insertions, deletions.

    alignres - AlignResults object
    polymer_length - int, minimum homopolymer length.
    bases - None, "AT" or "GC", type of homopolymers to look for
            None searches for all 
    specfic - Boole, if set to True, only hompolymers of the specified
              length will be analysed rather than the default of specified
              length or greater.
    """
    polymer_indexes = homopolymer_finder(alignres, polymer_length, bases)
    sequ = alignres.alignment[0]
    #Keeping only homopolymers of certain length.
    #TODO - clean this up
    if specific == True:
        i = 0
        while True:
            if i >= len(polymer_indexes):
                break
            if polymer_indexes[i][1]-polymer_indexes[i][0] != polymer_length-1:
                del polymer_indexes[i]
            else:
                i+=1
                
    #here errors follows [transitions,transverstions,
    #                     insertions, deletions]
    poly_errors = [0,0,0,0]
    post_errors = [0,0,0,0]
    poly_total, post_total = 0,0
    for i in polymer_indexes:
        for j in range(i[0], i[1]+1):
            poly_total += 1
            if j in alignres.index_transitions:
                poly_errors[0] += 1
            elif j in alignres.index_transversions:
                poly_errors[1] += 1
            #Technically this shouldn't be here as if it works
            #correctly you shouldn't be getting any insertions
            #mid polymer.
            elif j in alignres.index_insertions:
                poly_errors[2] += 1
            elif j in alignres.index_deletions:
                poly_errors[3] += 1

        post_total += 1
        if (i[1]+1) in alignres.index_transitions:
            post_errors[0] += 1
        elif (i[1]+1) in alignres.index_transversions:
            post_errors[1] += 1
        elif (i[1]+1) in alignres.index_insertions:
            post_errors[2] += 1
        elif (i[1]+1) in alignres.index_deletions:
            post_errors[3] += 1

    #Divide every element by the total
    if poly_total != 0:
        poly_errors[:] = [x / poly_total for x in poly_errors]
    if post_total != 0: 
        post_errors[:] = [x / post_total for x in post_errors]
    return (poly_errors, post_errors)

In [48]:
def homopolymer_analysis_multiple(results, polymer_length = 3, bases = None, specific = False):
    """
    A function that analyses the error rates due to homopolymers.
    It separately analyses error rates within the homopolymer and
    error rate directly one base downstream of the homopolymer.
    Returns two lists, [homopolymer_errors], [post-polymer errors]
    which each contain four variables representing the error rate
    of: transitions, transversions, insertions, deletions.

    results - A list of AlignRes objects.
    polymer_length - int, minimum homopolymer length.
    bases - None, "AT" or "GC", type of homopolymers to look for
            None searches for all.
    specfic - Boole, if set to True, only hompolymers of the specified
              length will be analysed rather than the default of specified
              length or greater.
    """
    
    poly_errors = [0,0,0,0]
    post_errors = [0,0,0,0]
    count = 0
    for i in results:
        count += 1
        if i != None:
            x,y = homopolymer_analysis(i, polymer_length, bases, specific)
            #Adding to existing errors
            for j in range(0,4):
                poly_errors[j] += x[j]
                post_errors[j] += y[j]

    if count == 0:
        return([0,0,0,0],[0,0,0,0])
    poly_errors[:] = [x / count for x in poly_errors]
    post_errors[:] = [x / count for x in post_errors]

    return(poly_errors, post_errors)

In [49]:
def repeat_finder(alignres, repeat_length = 3, repeat_count = 2):
    """
    A function that finds the indexes of all the
    repeats in a sequence.
    In this case the amount of repeats needing to occur
    is only 2, so "ATGATG" will be included.
    Returns a list of start and end indexes of each polymer.
    alginres - AlignResults object or a DNA sequence string.
    repeating_element - int, the size of the repeating element,
                        e.g., 2 = "ATAT" - "AT",
                              3 = "ATGATG" - "ATG".
                        Default = 3
    repeat_count - int, minimum times repeating element must
                   be encountered. E.g., if 3 then
                   "ATGATG" won't be taken into account but
                   "ATGATGATG" will. Default = 2
    """

    #Takes in AlignResults object or a pure sequence.
    if type(alignres) == AlignResults:
        sequ = alignres.alignment[0]
    else:
        sequ = alignres

    repeat_idx = []
    #This loop returns the starting index of each repeat.
    #This does result in overlaps for repeat sizes greater than 2.
    #But this is dealt with in the next loop
    for i in range(repeat_length):
        while (i+(repeat_length*2)) < len(sequ):
            x = sequ[i:i+(repeat_length*2)]
            pp = principal_period(x)
            if pp is not None:
                if len(pp) == repeat_length:
                    repeat_idx.append(i)
            i += repeat_length
            
    #Removing duplicating repeats
    #For example "ATG ATG ATG"
    #also contains "A TGATGA TG" "TGA" repeat
    #Solve this by checking for overlaps and deleting them
    repeat_idx.sort()
    i = 0
    while i < (len(repeat_idx)-1):
        if i == 0:
            i+=1
            continue
        if (repeat_idx[i] - repeat_idx[i-1]) < repeat_length:
            del repeat_idx[i]
            continue 
        i+=1

    #This here converts the starting index of each repeat
    #into a list of ranges as well as accounting for
    #repeats where the repeating elements occurs more than twice.
    combined_idx = []
    prev = False #Are we currently in series or not
    count = 0
    for i in range(len(repeat_idx)):
        #catch out of bounds
        if i == len(repeat_idx)-1:
            if prev == True:
                prev = False
                combined_idx.append([memory, repeat_idx[i]])
            break
        #If two repeats are in series:
        if (repeat_idx[i+1] - repeat_idx[i]) == repeat_length:
            if prev == False:
                count = 1
                prev = True
                memory = repeat_idx[i]
            else:
                count+=1
                continue
        else:
            if prev == True:
                prev = False
                if count >= (repeat_count-2):
                    combined_idx.append([memory, repeat_idx[i]+(repeat_length*2)-1])
                count = 0
            else:
                if repeat_count == 2:
                    combined_idx.append([repeat_idx[i], repeat_idx[i]+(repeat_length*2)-1])
                    count = 0

    return(combined_idx)

In [50]:
#This function is just a rehash of the above homopolymer function 
#that works with repeats instead

def repeat_analysis(alignres, repeat_length = 3, repeat_count = 2):
    """
    A function that analyses the error rates due to repeats.
    It separately analyses error rates within the repeat and
    error rate directly one base downstream of the repeat.
    Returns two lists, [repeat_errors], [post-repeat errors]
    which each contain four variables representing the error rate
    of: transitions, transversions, insertions, deletions.

    alignres - AlignResults object
    repeating_element - int, the size of the repeating element,
                        e.g., 2 = "ATAT" - "AT",
                              3 = "ATGATG" - "ATG".
                        Default = 3
    repeat_count - int, minimum times repeating element must
                   be encountered. E.g., if 3 then
                   "ATGATG" won't be taken into account but
                   "ATGATGATG" will. Default = 2
    """

    repeat_indexes = repeat_finder(alignres, repeat_length, repeat_count)
    sequ = alignres.alignment[0]

    #here errors follows [transitions,transverstions,
    #                     insertions, deletions]
    repeat_errors = [0,0,0,0]
    post_errors = [0,0,0,0]
    repeat_total, post_total = 0,0
    for i in repeat_indexes:
        for j in range(i[0], i[1]+1):
            repeat_total += 1
            if j in alignres.index_transitions:
                repeat_errors[0] += 1
            elif j in alignres.index_transversions:
                repeat_errors[1] += 1
            #Technically this shouldn't be here as if it works
            #correctly you shouldn't be getting any insertions
            #mid repeat.
            elif j in alignres.index_insertions:
                repeat_errors[2] += 1
            elif j in alignres.index_deletions:
                repeat_errors[3] += 1

        post_total += 1
        if (i[1]+1) in alignres.index_transitions:
            post_errors[0] += 1
        elif (i[1]+1) in alignres.index_transversions:
            post_errors[1] += 1
        elif (i[1]+1) in alignres.index_insertions:
            post_errors[2] += 1
        elif (i[1]+1) in alignres.index_deletions:
            post_errors[3] += 1

    if repeat_total == 0:
        repeat_errors = [0,0,0,0]
        post_errors = [0,0,0,0]
    else:
        #Divide every element by the total
        repeat_errors[:] = [x / repeat_total for x in repeat_errors]
        post_errors[:] = [x / post_total for x in post_errors]

    return (repeat_errors, post_errors)

In [51]:
def repeat_analysis_multiple(alignres_list, repeat_length = 3, repeat_count = 2):
    """
    A function that analyses the error rates due to repeats.
    It separately analyses error rates within the repeat and
    error rate directly one base downstream of the repeat.
    Returns two lists, [repeat_errors], [post-repeat errors]
    which each contain four variables representing the error rate
    of: transitions, transversions, insertions, deletions.

    alignres_list - List of AlignResults objects
    repeating_element - int, the size of the repeating element,
                        e.g., 2 = "ATAT" - "AT",
                              3 = "ATGATG" - "ATG".
                        Default = 3
    repeat_count - int, minimum times repeating element must
                   be encountered. E.g., if 3 then
                   "ATGATG" won't be taken into account but
                   "ATGATGATG" will. Default = 2
    """

    repeat_errors = [0,0,0,0]
    post_errors = [0,0,0,0]
    count = 0
    for i in alignres_list:
        count += 1
        if i != None:
            x,y = repeat_analysis(i, repeat_length, repeat_count)
            #Adding to existing errors
            for j in range(0,4):
                repeat_errors[j] += x[j]
                post_errors[j] += y[j]

    repeat_errors[:] = [x / count for x in repeat_errors]
    post_errors[:] = [x / count for x in post_errors]

    return(repeat_errors, post_errors)

In [52]:
def gc_sliding_window(alignres, windowsize = 100, step = 50, group = True, rate = False):
    """
    A function that performs a GC sliding window analysis on an alignment.
    For each window, it calculates GC content, as well as the number
    of transitions,transversion,insertsions,deletions or their rates
    if rate = True.
    Returns a list of lists. [GC content, [error rates]]
    alignres - An AlignResults object.
    windowsize - int, the bp size of the sliding window. Default = 100
               - float, relative size to the length of the alignment
    step - int, the bp size of each step of the window. I.e., how
           many bases the window moves by in each iteration.
         - float, relative size to the length of the alignment
           Default = 50
    group - boole, whether or not to treat indels greater than one bp in
            size as a single indel or multiple indels. Default = True
    rate - boole, whether to return the error rate per base instead of
           the number of errors. Default = False
    """

    #If given as a percentage
    if windowsize < 1:
        windowsize = int(windowsize*len(alignres))
    if step < 1:
        step = int(step*len(alignres))

    #These two values are needed to treat large indels as just one indel
    prev_ins, prev_del = False, False 
    gc_list = []
    i = 0
    #If group is True, "A---A" is one gap
    #If group is False, "A---A" is three gaps
    if group == True:
        while (i + windowsize) < len(alignres):
            frac = gc_fraction(alignres.alignment[0][i:i+windowsize])
            gc_errors = [0,0,0,0]
            for j in range(i, i+windowsize):
                if j in alignres.index_transitions:
                    gc_errors[0] += 1
                    prev_ins, prev_del = False, False 
                elif j in alignres.index_transversions:
                    gc_errors[1] += 1
                    prev_ins, prev_del = False, False 
                elif j in alignres.index_insertions:
                    if prev_ins == True:
                        continue
                    else:
                        gc_errors[2] += 1
                        prev_ins, prev_del = True, False 
                elif j in alignres.index_deletions:
                    if prev_del == True:
                        continue
                    else:
                        gc_errors[3] += 1
                        prev_ins, prev_del = False, True
            i += step
            if rate == True:
                gc_errors = (np.array(gc_errors) / windowsize).tolist()
            gc_list.append([frac, gc_errors])
    else:
        while (i + windowsize) < len(alignres):
            frac = gc_fraction(alignres.alignment[0][i:i+windowsize])
            gc_errors = [0,0,0,0]
            for j in range(i, i+windowsize):
                if j in alignres.index_transitions:
                    gc_errors[0] += 1
                elif j in alignres.index_transversions:
                    gc_errors[1] += 1
                elif j in alignres.index_insertions:
                    gc_errors[2] += 1
                elif j in alignres.index_deletions:
                    gc_errors[3] += 1
            i += step
            if rate == True:
                gc_errors = (np.array(gc_errors) / windowsize).tolist()
            gc_list.append([frac, gc_errors])

    return(gc_list)

In [53]:
def gc_grouper(gc_list, blocks = 5):
    """
    A function which groups the results from the
    gc_sliding_window function into a defined amount of blocks.
    For example if blocks = 5, the results from the function
    will be divided into 0.05 GC blocks. [[0-0.05, error rates],
    [0.05-0.10, error rates]...]
    gc_list - list, A list generated from the gc_sliding_window
                    function
    blocks - int, The size of each GC block.
    """

    final_list = []
    i = 0.00
    while i < 1:
        count = 0
        results = np.array([0,0,0,0],dtype='d')
        for j in gc_list:
            if j[0] < (i + (blocks/100)) and j[0] >= i:
                results += np.array(j[1])
                count += 1
        if count != 0:
            avg = (results / count).tolist()
            final_list.append([round(i,2), avg])
        else:
            final_list.append([round(i,2),[0,0,0,0]])

        i += (blocks/100)
    return(final_list)

In [54]:
def gc_sliding_window_multiple(alignres, windowsize = 100, step = 50, group = True, rate = False):
    """
    A function that performs a GC sliding window analysis on a list 
    of alignments. For each window, it calculates GC content, as 
    well as the number of transitions,transversion,insertsions,deletions
    or their rates if rate = True.
    Returns a list of lists. [GC content, [error rates]]
    alignres - An AlignResults object.
    windowsize - int, the bp size of the sliding window. Default = 100
               - float, relative size to the length of the alignment
    step - int, the bp size of each step of the window. I.e., how
           many bases the window moves by in each iteration.
         - float, relative size to the length of the alignment
           Default = 50
    group - boole, whether or not to treat indels greater than one bp in
            size as a single indel or multiple indels. Default = True
    rate - boole, whether to return the error rate per base instead of
           the number of errors. Default = False
    """
    final = []
    for i in alignres:
        if i != None:
            final += (gc_sliding_window(i, windowsize, step, group, rate))
    return final

In [55]:
#TODO: Incredibly messy can be written much better.

def area_creator(lst_idx):
    """
    A function that takes in a list of indexes and converts
    them into a list of lists, grouping runs (gap of 1) together.
    For example, [1,2,3,5,8,9] becomes:
    [[1,3],[5],[8,9]]
    """

    final = []
    series = False #whether the indexes are one after another
    memory = None #record the first element of the series
    for i in range(len(lst_idx)):
        if i == 0: #skipping first index as comparisons are against -1
            continue
        if i == len(lst_idx)-1: #Separate section for last index needed
            if series == True:
                series = False
                if (lst_idx[i] - lst_idx[i-1]) == 1:
                    if len(final) > 0:
                        if memory == final[-1][0]:
                            final.pop(-1)
                    final.append([memory,lst_idx[i]])
                else:
                    final.append([lst_idx[i]])
        if (lst_idx[i] - lst_idx[i-1]) == 1:
            if series == False:
                memory = lst_idx[i-1]
                series = True
            series = True
            continue
        else:
            if series == True:
                series = False
                if len(final) > 0: #removal of duplicates
                    if memory == final[-1][0]:
                        final.pop(-1)
                final.append([memory,lst_idx[i-1]])
                final.append([lst_idx[i]])
            else:
                final.append([lst_idx[i]])
    return final

------------------

### Q-score analysis in one function

In [56]:
def gc_grouper_qscore(gc_list, blocks = 5):
    """
    A function which groups the results from the
    gc_sliding_window function into a defined amount of blocks.
    For example if blocks = 5, the results from the function
    will be divided into 0.05 GC blocks. [[0-0.05, Q-score],
    [0.05-0.10, Q-scire]...]
    gc_list - list, A list generated from the gc_sliding_window
                    function
    blocks - int, The size of each GC block.
    """

    final_list = []
    i = 0.00
    while i < 1:
        count = 0
        results = 0
        for j in gc_list:
            if j[0] < (i + (blocks/100)) and j[0] >= i:
                results += j[1]
                count += 1
        if count != 0:
            avg = (results / count)
            final_list.append([round(i,2), avg])
        else:
            final_list.append([round(i,2),0])

        i += (blocks/100)
    return(final_list)

Below is the main function used to generate the data used for the analysis. It generates many results at once.

In [57]:
def avg_qscore_analysis(filename, show = True, plength = 3, p_downstream = 0, skip = 0, trim = 0,
                        windowsize = 100, step = 50, repeat_length = 3, ignore = 0, r_downstream = 0, limit = 0):
    """
    The main q-score analysis function.
    Takes in a fastq file.
    Output: Average Q-score, GC homopolymer Q-score, AT homopolymer Q-score,
            All hompolymer Q-score, Repeat (size 2) Q-score,
            Repeat (size 3) Q-score.
    filename - str, filename/location of fastq file.
    show - boole, whether to print the results. Default = True
    plength - int, minimum homopolymer length. Default = 3
    p_downstream - int, how many bases downstream of a homopolymer
                   to be included in the homopolymer analysis.
                   Default = 0
    skip - int, how many bases to exclude from the homopolymer analysis
           beginning from the start of the homopolymer. Default = 0
    trim - float, proportion of the start and end of each read to be
           trimmed, mainly for testing. Default = 0
    windowsize - int, the base size of the window for GC analysis.
                 Default = 100
    step - int, the step size for each shift of the GC window.
           Default = 50.
    repeat_length - int, the minimum amount of repepating elements
                    needed to occur to be classified as either a 
                    dimer or trimer repeat. Default = 3
    ignore - int, how many bases to ignore at the start of the repeat. For 
             example, a value of 2, with "ATGATG", only "GATG" is counted.
             Defualt = 0
    r_downstream - int, how many bases downstream of a trimer or dimer
                   repeat to be included in the repeat analysis.
                   Default = 0.
    limit - int, the minimum base size (larger than) that a read must be,
            otherwise it is ignored.
            Default = 0.
    """

    datas = []
    for seq_record in SeqIO.parse(filename, "fastq"):
        datas.append(seq_record)
    polymer_GC, polymer_AT, avg1 = [],[],[]
    gc, repeat_3, repeat_2 = [],[],[]
    for i in datas:
        if len(i) > limit:
            x,y = polymer_avg_qscore(i, plength, p_downstream, skip, trim, type = "GC")
            polymer_GC.append(x)
            avg1.append(y)
            x,y = polymer_avg_qscore(i, plength, p_downstream, skip, trim, type = "AT")
            polymer_AT.append(x)
            avg1.append(y)
            gc.append(analyse_gc_qscore(i, windowsize, step, trim))
            repeat_3.append(find_repeats_qscore(i, repeat_length, ignore, r_downstream, trim))
            repeat_2.append(find_repeats_qscore(i, 2, ignore, r_downstream, trim))

    if show == True:
        print('Results:')
        print('Average Q-score: ', (sum(avg1) / len(avg1)))
        print('GC Homopolymer Q-score: ', (sum(polymer_GC)/len(polymer_GC)))
        print('AT Homopolymer Q-score: ', (sum(polymer_AT)/len(polymer_AT)))
        print('All Homopolymer Q-score: ', ((sum(polymer_GC) + sum(polymer_AT)) / (len(polymer_GC) + len(polymer_AT))))
        print('Repeat Q-score (size 3): ', (sum(repeat_3) / len(repeat_3)))
        print('Repeat Q-score (size 2): ', (sum(repeat_2) / len(repeat_2)))
        
    return([(sum(avg1) / len(avg1)), (sum(polymer_GC)/len(polymer_AT)),(sum(polymer_AT)/len(polymer_AT)),
            ((sum(polymer_GC) + sum(polymer_AT)) / (len(polymer_GC) + len(polymer_AT))),
            (sum(repeat_3) / len(repeat_3)), (sum(repeat_2) / len(repeat_2)), gc])

Below is an example as to how the analysis was performed and how the data was exported into csv files using pandas.

In [60]:
first = True
count = 1
for i in ["NextSeq_Human/ERR9772230.fastq","NextSeq_Human/ERR9772276.fastq",
          "NextSeq_Human/SRR27883880.fastq"]:
    #To name files automaitcally
    name, write = "", False
    for j in i:
        if j == "/":
            write = True
            continue
        if j == ".":
            break
        if write == True:
            name += j
    name += "_GC_Q.csv"
    print(name)
    if first == True:
        x = avg_qscore_analysis(i, limit = 50)
        first = False
        avg_q = pd.DataFrame([[name] + x[0:6]], columns = ["Run","Avg_Q_score","GC_polymer","AT_polymer","All_polymer","Repeat_Size_2","Repeat_Size_3"])
        y = []
        for i in x[6]:
            y += i
        y = gc_grouper_qscore(y,5)
        gc_q_df = pd.DataFrame(y, columns = ["GC", "Q_score"])
        gc_q_df.to_csv(name, encoding = 'utf-8', index = False)
    else:
        x = avg_qscore_analysis(i, limit = 50)
        avg_q.loc[count] = [name] + x[0:6]
        count += 1
        y = []
        for i in x[6]:
            y += i
        y = gc_grouper_qscore(y,5)
        gc_q_df = pd.DataFrame(y, columns = ["GC", "Q_score"])
        gc_q_df.to_csv(name, encoding = 'utf-8', index = False)

avg_q.to_csv("Avg_Q_NextSeq_Human.csv", encoding = 'utf-8', index = False)

ERR9772230_GC_Q.csv
Results:
Average Q-score:  32.640651041187944
GC Homopolymer Q-score:  24.949135939432747
AT Homopolymer Q-score:  29.81854476736324
All Homopolymer Q-score:  27.383840353397993
Repeat Q-score (size 3):  21.57360551058299
Repeat Q-score (size 2):  31.069804905809256
ERR9772276_GC_Q.csv
Results:
Average Q-score:  31.070080935995822
GC Homopolymer Q-score:  27.570888953801735
AT Homopolymer Q-score:  24.85735097816374
All Homopolymer Q-score:  26.214119965982736
Repeat Q-score (size 3):  18.128789708046245
Repeat Q-score (size 2):  30.082885578733812
SRR27883880_GC_Q.csv
Results:
Average Q-score:  24.91477623316758
GC Homopolymer Q-score:  18.912241297956516
AT Homopolymer Q-score:  22.480942252549816
All Homopolymer Q-score:  20.696591775253168
Repeat Q-score (size 3):  15.265493085550434
Repeat Q-score (size 2):  23.3963674639759


-------------

### Indel/substitution analysis in one function

This function here was introduced late and aimed to analyses whther the position of the read affected the error rate. \
Blocks below is into howmany segments the read is divided into. \
100 means 0-1%, 1-2%, 2-3%...

In [61]:
def regional_analysis(alignres, blocks = 100, group = True):

    size = len(alignres) / blocks
    prev_ins, prev_del = False, False 
    error_list = []
    i,region = 0, 0
    #If group is True, "A---A" is one gap
    #If group is False, "A---A" is three gaps
    if group == True:
        while (i + size) < len(alignres):
            errors = [0,0,0,0]
            x = int(round(i + size))
            for j in range(i, x):
                if j in alignres.index_transitions:
                    errors[0] += 1
                    prev_ins, prev_del = False, False 
                elif j in alignres.index_transversions:
                    errors[1] += 1
                    prev_ins, prev_del = False, False 
                elif j in alignres.index_insertions:
                    if prev_ins == True:
                        continue
                    else:
                        errors[2] += 1
                        prev_ins, prev_del = True, False 
                elif j in alignres.index_deletions:
                    if prev_del == True:
                        continue
                    else:
                        errors[3] += 1
                        prev_ins, prev_del = False, True
            i += size
            i = int(round(i))
            region += 1/blocks
            region = round(region,3)
            errors = (np.array(errors) / size).tolist()
            error_list.append(errors)
    else:
        while (i + size) < len(alignres):
            errors = [0,0,0,0]
            for j in range(i, int(round(i+size))):
                if j in alignres.index_transitions:
                    errors[0] += 1
                elif j in alignres.index_transversions:
                    errors[1] += 1
                elif j in alignres.index_insertions:
                    errors[2] += 1
                elif j in alignres.index_deletions:
                    errors[3] += 1
            i += size
            i = int(round(i))
            region += 1/blocks
            region = round(region,3)
            errors = (np.array(errors) / size).tolist()
            error_list.append(errors)

    while len(error_list) < blocks:
        error_list.append([0,0,0,0])
    while len(error_list) > blocks:
        del error_list[-1]
    return(np.array(error_list))

In [62]:
def regional_analysis_multiple(res_list, blocks = 100, group = True):

    first = True
    count = 0
    for i in res_list:
        if i != None:
            count += 1
            if first:
                errors = regional_analysis(i, blocks, group)
                first = False
            else:
                errors += regional_analysis(i,blocks,group)
    errors = errors / count
    return(errors)
            

In [63]:
def len_value_error_alignment(x):
    '''
    A function to deal with empty alignments when comparing
    lengths
    '''
    try:
        len(x[0])
    except ValueError:
        return 0
    else:
        return len(x[0])

In [64]:
def size_analysis(filename, filetype = "sam", drop_last_insertion = True, drop_limit = 20, cutoff = 0.2, limit = 50):

    alignments = Align.parse(filename, filetype)
    align_list = parse_iterator(alignments)
    if limit != 0:
        align_list = [x for x in align_list if len_value_error_alignment(x) > limit]
    res_list = []
    for i in align_list:
        res_list.append(align_analysis(i, cutoff = cutoff))
        
    insertions = {}
    deletions = {}
    for i in res_list:
        if i != None:
            temp = indel_size_analysis(i, drop_last_insertion, drop_limit)
            for j in temp[0]:
                if j not in insertions:
                    insertions[j] = 1
                else:
                    insertions[j] += 1
            for j in temp[1]:
                if j not in deletions:
                    deletions[j] = 1
                else:
                    deletions[j] += 1
    return(insertions, deletions)

In [65]:
def dictionary_combiner(dic1, dic2):

    for i in dic2:
        if i not in dic1:
            dic1[i] = dic2[i]
        else:
            dic1[i] += dic2[i]
    return(dic1)

In [66]:
def size_dict_df(dic):

    final_list = []
    for i in sorted(dic):
        final_list.append([i, dic[i]])
    final_df = pd.DataFrame(final_list, columns = ["Size", "Count"])
    return(final_df)

Below is the main function for analysis. Outputs pandas dataframes that can later be converted into csv files

In [67]:
def indel_substitution_analysis(filename, filetype = "sam", group = True, polymer_length = 3, repeat_length = 3,
                                repeat_count = 3, windowsize = 100, step = 50, rate = True, gc_blocks = 5,
                                reg_blocks = 10, cutoff = None, limit = 0):
    """
    outputs three dataframes; regular errors, gc errors and regional errors
    filename - str, name/location of file
    filetype - "sam" or "bam", type of alignment file used.
    group - True or False, whether to count large indels as one. E.g
            If true "A---A" counts as 1 gap, If false -> 3 gaps.
            Default = True
    polymer_length - int, minimum homopolymer length. Default = 3
    repeat_length - int or list, the size fo the repeating element.
                    Default = 3
    repeat_count - int or list, minimum number of repeating elements to be
                    considered a repeat. Default = 3
    windowsize - int, the base size of the window for GC analysis. Default = 100
    step - int, the step size for the sliding GC window. Default = 50
    rate - boole, whether to return the error rate per base instead of
           the number of errors. Default = True
    gc_blocks - int, the size of the blocks that GC is grouped into.
                E.g., if 5 then 0.01, 0.02... 0.05, will all be grouped as 0.05.
                Default = 5.
    reg_blocks - int, the size of each block for comparing regional error rates.
                 Default = 10
    cutoff - float (0-1), if error rate is higher than cuttoff, it is ignored.
             Default = 0 
    limit - int, the minimum base size (larger than) that a read must be,
            otherwise it is ignored.
            Default = 0.
    """
    alignments = Align.parse(filename, filetype)
    align_list = parse_iterator(alignments)
    if limit != 0:
        align_list = [x for x in align_list if len_value_error_alignment(x) > limit]
    res_list = []
    for i in align_list:
        res_list.append(align_analysis(i, cutoff = cutoff))

    flat_errors = flat_error_analysis(res_list, group)
    polymer = homopolymer_analysis_multiple(res_list, polymer_length, bases = None)
    polymer_AT = homopolymer_analysis_multiple(res_list, polymer_length, bases = "AT")
    polymer_GC = homopolymer_analysis_multiple(res_list, polymer_length, bases = "GC")

    df = pd.DataFrame(np.array([["flat"] + flat_errors, ["all_polymers"] + polymer[0], ["post_all_polymer"] + polymer[1],
                               ["AT_polymers"] + polymer_AT[0], ["post_AT_polymers"] + polymer_AT[1],
                               ["GC_polymers"] + polymer_GC[0], ["post_GC_polymers"] + polymer_GC[1]]),
                                columns=['name','transitions', 'transversions', 'insertions','deletions'])
       
    if type(repeat_length) == list:
        for i in range(len(repeat_length)):
            x = repeat_analysis_multiple(res_list, repeat_length[i], repeat_count[i])
            df.loc[len(df.index)] = ["Repeat " + str(repeat_length[i]) + ", " + str(repeat_count[i])] + x[0]
            df.loc[len(df.index)] = ["Post_Repeat " + str(repeat_length[i]) + ", " + str(repeat_count[i])] + x[1]
    else:
        x = repeat_analysis_multiple(res_list, repeat_length, repeat_count)
        df.loc[len(df.index)] = ["Repeat " + str(repeat_length) + ", " + str(repeat_count)] + x[0]
        df.loc[len(df.index)] = ["Post_Repeat " + str(repeat_length) + ", " + str(repeat_count)] + x[1]

    gc2 = []
    gc = gc_grouper(gc_sliding_window_multiple(res_list, windowsize, step, group, rate), gc_blocks)
    # converting to a single list instead of having gc content and error rates as separate lists
    for i in gc:
        gc2.append([i[0]] + i[1])
    gc_df = pd.DataFrame(gc2, columns = ["GC_content", "transitions", "transversions", "insertions", "deletions"])
    
    regional = regional_analysis_multiple(res_list, reg_blocks, group)
    regional_blocks = [(i/reg_blocks + 1/reg_blocks) for i in range(reg_blocks)]
    reg_df = pd.DataFrame(regional, columns = ['transitions', 'transversions', 'insertions','deletions'])  
    reg_df.insert(0, "block", regional_blocks, True)
    return(df, gc_df, reg_df)

Below is a function that simply looks at how homopolymer length affects error rate.

In [68]:
#Creating another function as the first one is getting to messy
#Parsing the alignment isn't very computationally expensive so 
#it's not too bad to have them separate.
def homopolymer_length_analysis(filename, filetype = "sam", polymer_length = [3,12], bases = None, 
                                specific = False, cutoff = None, limit = 0):

    alignments = Align.parse(filename, filetype)
    align_list = parse_iterator(alignments)
    if limit != 0:
        align_list = [x for x in align_list if len_value_error_alignment(x) > limit]
    res_list = []
    for i in align_list:
        res_list.append(align_analysis(i, cutoff = cutoff))

    pol_list = []
    for i in range(polymer_length[0],polymer_length[1]+1):
        #Ignoring post errors for this one, however insertion rate
        # in post errors is technically the actual insertion rate in polymer.
        x,y = homopolymer_analysis_multiple(res_list, i, bases, specific)
        x[2] = y[2]
        pol_list.append([i] + x)

    pol_df = pd.DataFrame(pol_list, columns = ["homopolymer_size","transitions", "transversions", "insertions", "deletions"])
    return(pol_df)

In [69]:
def add_sum_errors(df):
    sums = []
    for i in range(df.shape[0]):
        sums.append(sum(df.iloc[i][1:5]))
    df.insert(5, "total_error_rate", sums, True)
    return(df)

Below is an example of how the above functions are used.

In [72]:
ins_size_list, del_size_list = [],[]
for i in ["NextSeq_Human/ERR9772230.sam","NextSeq_Human/ERR9772276.sam","NextSeq_Human/SRR24436571.sam",
          "NextSeq_Human/SRR27883880.sam","NextSeq_Human/SRR28293697.sam"]:
    print(i)
    sizes = size_analysis(i, cutoff = 0.2, limit = 50)
    ins_size_list.append(sizes[0])
    del_size_list.append(sizes[1])
    analyses = indel_substitution_analysis(i, repeat_length = [3,2], repeat_count = [3,3], cutoff = 0.2, limit = 50)
    pol_analysis = homopolymer_length_analysis(i, specific = False, limit = 50, cutoff = 0.2)
    
    
    name, write = "", False
    #To name files automaitcally
    for j in i:
        if j == "/":
            write = True
            continue
        if j == ".":
            break
        if write == True:
            name += j
            
    analyses[0].to_csv(name + "_errors.csv", encoding='utf-8', index=False)
    analyses[1].to_csv(name + "_GC.csv", encoding='utf-8', index=False)
    analyses[2].to_csv(name + "_regions.csv", encoding='utf-8', index=False)
    pol_analysis.to_csv(name + "_hompolymers.csv", encoding='utf-8', index=False)
    

insertions = ins_size_list[0]
for i in ins_size_list[1:]:
    insertions = dictionary_combiner(insertions,i)
deletions = del_size_list[0]
for i in del_size_list[1:]:
    deletions = dictionary_combiner(deletions, i)

size_dict_df(insertions).to_csv("NextSeq_Human_Insertion_Sizes.csv", encoding='utf-8', index=False)
size_dict_df(deletions).to_csv("NextSeq_Human_Deletion_Sizes.csv", encoding='utf-8', index = False)

NextSeq_Human/ERR9772230.sam
NextSeq_Human/ERR9772276.sam
NextSeq_Human/SRR24436571.sam
NextSeq_Human/SRR27883880.sam
NextSeq_Human/SRR28293697.sam


In [ ]:
#df.to_csv(file_name, encoding='utf-8', index=False)

In [ ]:
#test[2].to_csv("test.csv", encoding='utf-8', index=False)

### 